# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#!pip install gmaps

# enable jupyter extensions
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# install gmaps
#!conda install -c conda-forge gmaps --yes
# enable gmaps
#!jupyter nbextension enable --py --sys-prefix gmaps 




import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather="Instructions/output_data/City.csv"

city_weather_df=pd.read_csv(city_weather)

city_weather_df.head()




### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)



humidity = city_weather_df["Humidity"].astype(float)
max_hum = humidity.max()
latlng=city_weather_df[["Lat", "Lng"]]

figure = gmaps.figure()
heat = gmaps.heatmap_layer(latlng, weights = humidity)
figure.add_layer(heat)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
city_df = city_weather_df.loc[city_weather_df["Max Temp"]< 80, :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df["index"]
city_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.52,-28.70,77.00,83,20,8.05,PT,1595870936
1,Isetskoye,56.49,65.32,59.31,78,22,5.93,RU,1595870936
2,Cape Town,-33.93,18.42,59.00,82,0,8.05,ZA,1595870818
3,Oberá,-27.49,-55.12,67.93,78,63,1.68,AR,1595870937
4,Busselton,-33.65,115.33,51.01,98,18,8.08,AU,1595871401


In [8]:
hotels = []

for i in range(len(city_df)):
    lat = city_df.loc[i]["Lat"]
    lng = city_df.loc[i]["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    json_request = request.json()
    try:
        hotels.append(json_request["results"][0]["name"])
    except:
        hotels.append("")
        

city_df["Hotel Name"] = hotels
hotel_df = city_df.dropna(how='any')
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ribeira Grande,38.52,-28.70,77.00,83,20,8.05,PT,1595870936,Horta
1,Isetskoye,56.49,65.32,59.31,78,22,5.93,RU,1595870936,Isetskoye
2,Cape Town,-33.93,18.42,59.00,82,0,8.05,ZA,1595870818,Cape Town
3,Oberá,-27.49,-55.12,67.93,78,63,1.68,AR,1595870937,Oberá
4,Busselton,-33.65,115.33,51.01,98,18,8.08,AU,1595871401,Busselton


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
figure.add_layer(marker)

figure

# Display figure


Figure(layout=FigureLayout(height='420px'))